### QUESTION 13: 
Using grid search, find the best architecture (for scaled data) for each period (with corresponding window length) described in Question 6.

In [1]:
hash_tags = ['#gohawks','#gopatriots','#nfl','#patriots','#sb49','#superbowl']

In [2]:
import pickle

def save_object(data, fileName):
    with open('pynb_data/'+fileName + ".pickle", 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
        
def load_object(fileName):
    try:
        with open('pynb_data/'+fileName + ".pickle", 'rb') as f:
            data = pickle.load(f)
            return data
    except IOError:
        print("Could not read file: " + fileName)

In [3]:
import json

def getMinAndMaxTs(tag):
    filename = 'data/tweets_'+tag+'.txt'
    max_ts = 0
    min_ts = 1552522378
    with open(filename) as f:
        for line in f:
            json_object = json.loads(line)
            timestamp = json_object['citation_date']
            if(timestamp < min_ts):                
                min_ts = timestamp
            
            if(timestamp > max_ts):
                max_ts = timestamp
                
    return [min_ts,max_ts]

tagsToMinTs = {}
tagsToMaxTs = {}
globalMinTs = 1552522378
globalMaxTs = 0
for tag in hash_tags:
    ts_list = getMinAndMaxTs(tag)
    
    if(ts_list[0]<globalMinTs):
        globalMinTs = ts_list[0]
    
    if(ts_list[1]>globalMaxTs):
        globalMaxTs = ts_list[1]
        
    tagsToMinTs[tag] = (ts_list[0])
    tagsToMaxTs[tag] = (ts_list[1])    

In [4]:
import math
import datetime
import pytz


def getLocalHour(timestamp):
    d = datetime.datetime.fromtimestamp(timestamp)
    pst = pytz.timezone('America/Los_Angeles')
    d = pst.localize(d)
    return d.hour

def getWindowNumber(start_ts, curr_ts, window):
    elapsed = (curr_ts - start_ts)/(window*1.0)
    windowNum = math.ceil(elapsed)
    return windowNum    

def getFeatures(start_ts,end_ts,window):
    windowToTweets = {}
    windowToRetweets = {}
    windowToFollowerCount = {}
    windowToMaxFollowers = {}
    features = []
    labels = []
    
    for tag in hash_tags:
        filename = 'data/tweets_'+tag+'.txt'
        with open(filename) as f:
            for line in f:
                json_object = json.loads(line)
                timestamp = json_object['citation_date']
            
                if timestamp < start_ts or timestamp > end_ts:                            
                    continue
                
                key = getWindowNumber(start_ts,timestamp,window)
    #             print(key)
                if key not in windowToTweets.keys():
                    windowToTweets[key]=0
                windowToTweets[key]+=1
            
                retweetCount = json_object['metrics']['citations']['total']        
            
                if key not in windowToRetweets.keys():
                    windowToRetweets[key]=0
                windowToRetweets[key]+=retweetCount
        
                followerCount = json_object['author']['followers']
                if key not in windowToFollowerCount.keys():
                    windowToFollowerCount[key]=0
                windowToFollowerCount[key]+=followerCount
        
                if key not in windowToMaxFollowers.keys():
                    windowToMaxFollowers[key]=0
                windowToMaxFollowers[key] = max(windowToMaxFollowers[key],followerCount)            
            
    for period in range(start_ts,end_ts,window):
        key = getWindowNumber(start_ts,period,window)
        tweetCount = windowToTweets.get(key, 0)
        retweetCount = windowToRetweets.get(key,0)
        followerCount = windowToFollowerCount.get(key,0)
        maxFollowers = windowToMaxFollowers.get(key,0)

        h = getLocalHour(period)
            
        feature = [tweetCount, retweetCount, followerCount, maxFollowers, h]
        features.append(feature)
                
        nextKey = getWindowNumber(start_ts, period + window, window)
        labels.append(windowToTweets.get(nextKey,0))
                
    return features,labels

In [5]:
# https://www.kaggle.com/grfiv4/displaying-the-results-of-a-grid-search
import numpy  as np
import pandas as pd

from sklearn.model_selection import GridSearchCV

def GridSearch_table_plot(grid_clf, param_name,
                          num_results=15,
                          negative=True,
                          graph=True,
                          display_all_params=True):

    from matplotlib      import pyplot as plt
    from IPython.display import display
    import pandas as pd

    clf = grid_clf.best_estimator_
    clf_params = grid_clf.best_params_
    if negative:
        clf_score = -grid_clf.best_score_
    else:
        clf_score = grid_clf.best_score_
    clf_stdev = grid_clf.cv_results_['std_test_score'][grid_clf.best_index_]
    cv_results = grid_clf.cv_results_

    print("best parameters: {}".format(clf_params))
    print("best score:      {:0.5f} (+/-{:0.5f})".format(clf_score, clf_stdev))
    if display_all_params:
        import pprint
        pprint.pprint(clf.get_params())

    # pick out the best results
    # =========================
    scores_df = pd.DataFrame(cv_results).sort_values(by='rank_test_score')

    best_row = scores_df.iloc[0, :]
    if negative:
        best_mean = -best_row['mean_test_score']
    else:
        best_mean = best_row['mean_test_score']
    best_stdev = best_row['std_test_score']
    best_param = best_row['param_' + param_name]

    # display the top 'num_results' results
    # =====================================
    display(pd.DataFrame(cv_results) \
            .sort_values(by='rank_test_score').head(num_results))

### Timeperiod 1

In [6]:

#tp1
tp1_window_size = 3600 # 1 hour window size
tp1_start_ts = tp1_window_size * math.floor(globalMinTs/(tp1_window_size*1.0))
tp1_end_ts = 1422806400

features,labels = getFeatures(tp1_start_ts,tp1_end_ts,tp1_window_size)
save_object(features,"q13_tp1_features")
save_object(features,"q13_tp1_labels")
# features = load_object("q7_tp1_features")
# labels = load_object("q7_tp1_labels")
print("Finished Building feature vectors for time period 1")

Finished Building feature vectors for time period 1


In [7]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

hidden_layer_sizes = [(100,),
                      (100,100,),
                      (100,100,100,),
                      (100,100,100,100,100),
                      (200,),
                      (200,200,),
                      (200,200,200,),
                      (500,250,125,63,),
                      (800,700,600,500,400,300,200,100,),
                      (1000,100,10),                      
                     ]

param_grid = [
    {
        'hidden_layer_sizes' : hidden_layer_sizes,
    }
]


nn = MLPRegressor(hidden_layer_sizes=(50,50),max_iter=500)

gs = GridSearchCV(estimator=nn,
                  param_grid=param_grid,
                  cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')

gs = gs.fit(scaled_features, labels)
GridSearch_table_plot(gs, "hidden_layer_sizes", negative=False)

save_object(gs,"q13_tp1_grid")


/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarnin

/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarnin

best parameters: {'hidden_layer_sizes': (100, 100)}
best score:      -5530582.42437 (+/-6297055.51177)
{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100, 100),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}


/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,1.893389,0.327991,0.001435,0.000928,"(100, 100)","{'hidden_layer_sizes': (100, 100)}",-1.754607e+07,-4.994349e+05,-3.762964e+05,-5.027250e+06,...,-5.530582e+06,6.297056e+06,1,-1.428790e+06,-5.454295e+06,-5.473721e+06,-4.860542e+06,-4.449457e+06,-4.333361e+06,1.502321e+06
2,2.158631,0.039230,0.001059,0.000183,"(100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100)}",-1.664629e+07,-5.555947e+05,-1.007868e+06,-4.968935e+06,...,-5.554781e+06,5.830125e+06,2,-1.222437e+06,-4.306962e+06,-4.229953e+06,-4.058824e+06,-3.183437e+06,-3.400323e+06,1.160528e+06
5,3.345734,0.405255,0.000990,0.000116,"(200, 200)","{'hidden_layer_sizes': (200, 200)}",-1.753597e+07,-5.032169e+05,-3.835964e+05,-5.389366e+06,...,-5.599687e+06,6.288058e+06,3,-1.426022e+06,-5.407128e+06,-5.453109e+06,-4.794264e+06,-4.439776e+06,-4.304060e+06,1.488550e+06
9,5.852919,0.989177,0.001580,0.000132,"(1000, 100, 10)","{'hidden_layer_sizes': (1000, 100, 10)}",-1.754341e+07,-5.233398e+05,-4.389602e+05,-5.351260e+06,...,-5.616620e+06,6.276973e+06,4,-1.410669e+06,-5.311235e+06,-5.280036e+06,-4.861239e+06,-4.498116e+06,-4.272259e+06,1.461622e+06
6,4.981404,0.094761,0.001378,0.000139,"(200, 200, 200)","{'hidden_layer_sizes': (200, 200, 200)}",-1.621055e+07,-5.579871e+05,-3.988770e+06,-3.787801e+06,...,-5.930318e+06,5.358802e+06,5,-8.494048e+05,-3.097383e+06,-2.768327e+06,-3.602562e+06,-2.492746e+06,-2.562085e+06,9.327116e+05
4,1.028281,0.063728,0.001096,0.000380,"(200,)","{'hidden_layer_sizes': (200,)}",-2.348214e+07,-1.678482e+06,-1.381411e+06,-1.488580e+06,...,-6.713676e+06,8.528213e+06,6,-2.667799e+06,-7.780859e+06,-8.046034e+06,-7.834203e+06,-6.876658e+06,-6.641111e+06,2.026771e+06
0,0.933394,0.157777,0.001142,0.000704,"(100,)","{'hidden_layer_sizes': (100,)}",-2.541408e+07,-2.230134e+06,-2.088038e+06,-2.630511e+06,...,-7.713594e+06,8.979260e+06,7,-3.383874e+06,-9.030890e+06,-9.165785e+06,-8.880178e+06,-7.956211e+06,-7.683387e+06,2.191156e+06
8,27.749458,5.414528,0.010789,0.000307,"(800, 700, 600, 500, 400, 300, 200, 100)","{'hidden_layer_sizes': (800, 700, 600, 500, 40...",-1.717340e+07,-3.683981e+05,-1.020491e+07,-1.376699e+07,...,-9.417730e+06,5.938744e+06,8,-1.387656e+05,-7.834173e+05,-5.139332e+05,-4.761908e+05,-3.336676e+05,-4.491949e+05,2.127615e+05
7,9.736174,0.848697,0.002127,0.000289,"(500, 250, 125, 63)","{'hidden_layer_sizes': (500, 250, 125, 63)}",-1.693986e+07,-3.555552e+05,-1.203666e+07,-1.863583e+07,...,-1.088063e+07,6.763618e+06,9,-2.688424e+05,-1.200764e+06,-5.997779e+05,-4.944560e+05,-1.038865e+06,-7.205411e+05,3.468932e+05
3,3.787855,0.247993,0.001478,0.000382,"(100, 100, 100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100, 100, 100)}",-1.691262e+07,-3.739770e+05,-9.949472e+06,-2.200424e+07,...,-1.105215e+07,7.679206e+06,10,-1.575809e+05,-4.397577e+05,-5.858708e+05,-3.300075e+05,-3.132020e+05,-3.652838e+05,1.423393e+05


In [8]:
# GridSearch_table_plot(gs, "hidden_layer_sizes", negative=False)


### Timeperiod 2:

In [9]:
#tp2
tp2_window_size = 300 # 5 minute window size
tp2_start_ts = 1422806400
tp2_end_ts = 1422849600
features,labels = getFeatures(tp2_start_ts,tp2_end_ts,tp2_window_size)
save_object(features,"q13_tp2_features")
save_object(labels,"q13_tp2_labels")
print("Finished Building feature vectors for time period 2")

Finished Building feature vectors for time period 2


In [10]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

hidden_layer_sizes = [(100,),
                      (100,100,),
                      (100,100,100,),
                      (100,100,100,100,100),
                      (200,),
                      (200,200,),
                      (200,200,200,),
                      (500,250,125,63,),
                      (800,700,600,500,400,300,200,100,),
                      (1000,100,10),                      
                     ]

param_grid = [
    {
        'hidden_layer_sizes' : hidden_layer_sizes,
    }
]


# nn = MLPRegressor(hidden_layer_sizes=(50,50), solver='adam')
nn = MLPRegressor(hidden_layer_sizes=(50,50),max_iter=500)

gs = GridSearchCV(estimator=nn,
                  param_grid=param_grid,
                  cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')

gs = gs.fit(scaled_features, labels)

GridSearch_table_plot(gs, "hidden_layer_sizes", negative=False)

save_object(gs,"q13_tp2_grid")

/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarnin

/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarnin

best parameters: {'hidden_layer_sizes': (100, 100, 100, 100, 100)}
best score:      -17991600.44716 (+/-7767897.40303)
{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100, 100, 100, 100, 100),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}


/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/rishabketandos

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,1.514844,0.009022,0.000924,0.000164,"(100, 100, 100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100, 100, 100)}",-1.564140e+07,-3.330446e+07,-1.339350e+07,-1.254775e+07,...,-1.799160e+07,7.767897e+06,1,-1.175093e+07,-9.501884e+06,-1.255605e+07,-1.367160e+07,-1.293279e+07,-1.208265e+07,1.431224e+06
7,3.718261,0.128429,0.001398,0.000055,"(500, 250, 125, 63)","{'hidden_layer_sizes': (500, 250, 125, 63)}",-1.466581e+07,-3.174865e+07,-1.237783e+07,-1.828410e+07,...,-1.874278e+07,6.821912e+06,2,-1.459305e+07,-1.048447e+07,-1.378848e+07,-1.572554e+07,-1.437428e+07,-1.379316e+07,1.769694e+06
6,2.057488,0.049719,0.001068,0.000053,"(200, 200, 200)","{'hidden_layer_sizes': (200, 200, 200)}",-1.872089e+07,-3.101676e+07,-1.273904e+07,-2.105624e+07,...,-2.060747e+07,5.941440e+06,3,-1.697089e+07,-1.446951e+07,-1.841590e+07,-1.821543e+07,-1.725251e+07,-1.706485e+07,1.409183e+06
9,2.869999,0.319238,0.001165,0.000142,"(1000, 100, 10)","{'hidden_layer_sizes': (1000, 100, 10)}",-2.479300e+07,-3.914060e+07,-3.227676e+07,-2.511212e+07,...,-2.889670e+07,6.047475e+06,4,-2.501002e+07,-1.955273e+07,-2.934452e+07,-2.338332e+07,-2.124366e+07,-2.370685e+07,3.373731e+06
8,32.032807,3.011274,0.005867,0.000487,"(800, 700, 600, 500, 400, 300, 200, 100)","{'hidden_layer_sizes': (800, 700, 600, 500, 40...",-1.791942e+07,-3.971804e+07,-3.960298e+07,-2.635563e+07,...,-2.900236e+07,9.160149e+06,5,-9.884797e+05,-5.714538e+05,-1.566739e+06,-5.439147e+05,-6.410590e+05,-8.623292e+05,3.865394e+05
2,0.909948,0.006428,0.000905,0.000030,"(100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100)}",-2.520822e+07,-3.999251e+07,-2.629018e+07,-2.920496e+07,...,-2.989490e+07,5.284894e+06,6,-2.440510e+07,-2.099525e+07,-2.506494e+07,-2.513934e+07,-2.603111e+07,-2.432715e+07,1.744425e+06
5,1.237142,0.019947,0.000794,0.000115,"(200, 200)","{'hidden_layer_sizes': (200, 200)}",-6.327808e+07,-6.126068e+07,-6.444263e+07,-5.036499e+07,...,-6.199717e+07,6.668754e+06,7,-5.739243e+07,-5.327603e+07,-5.585644e+07,-6.139151e+07,-5.335935e+07,-5.625515e+07,3.003177e+06
1,0.642481,0.020932,0.000871,0.000130,"(100, 100)","{'hidden_layer_sizes': (100, 100)}",-9.745093e+07,-9.729750e+07,-6.002519e+07,-5.991394e+07,...,-8.987762e+07,2.831926e+07,8,-8.683199e+07,-8.497857e+07,-9.775006e+07,-8.724492e+07,-8.659754e+07,-8.868062e+07,4.599519e+06
4,0.440291,0.003753,0.000720,0.000029,"(200,)","{'hidden_layer_sizes': (200,)}",-3.013237e+08,-3.886955e+08,-2.102267e+08,-2.159692e+08,...,-2.934938e+08,7.169487e+07,9,-2.913843e+08,-2.699043e+08,-3.148960e+08,-3.122993e+08,-2.786475e+08,-2.934263e+08,1.784942e+07
0,0.345393,0.009715,0.000769,0.000306,"(100,)","{'hidden_layer_sizes': (100,)}",-3.054345e+08,-3.944775e+08,-2.136767e+08,-2.198120e+08,...,-2.977657e+08,7.236567e+07,10,-2.955580e+08,-2.737516e+08,-3.191522e+08,-3.174670e+08,-2.829027e+08,-2.977663e+08,1.815481e+07


In [11]:
GridSearch_table_plot(gs, "hidden_layer_sizes", negative=False)


best parameters: {'hidden_layer_sizes': (100, 100, 100, 100, 100)}
best score:      -17991600.44716 (+/-7767897.40303)
{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100, 100, 100, 100, 100),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,1.514844,0.009022,0.000924,0.000164,"(100, 100, 100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100, 100, 100)}",-1.564140e+07,-3.330446e+07,-1.339350e+07,-1.254775e+07,...,-1.799160e+07,7.767897e+06,1,-1.175093e+07,-9.501884e+06,-1.255605e+07,-1.367160e+07,-1.293279e+07,-1.208265e+07,1.431224e+06
7,3.718261,0.128429,0.001398,0.000055,"(500, 250, 125, 63)","{'hidden_layer_sizes': (500, 250, 125, 63)}",-1.466581e+07,-3.174865e+07,-1.237783e+07,-1.828410e+07,...,-1.874278e+07,6.821912e+06,2,-1.459305e+07,-1.048447e+07,-1.378848e+07,-1.572554e+07,-1.437428e+07,-1.379316e+07,1.769694e+06
6,2.057488,0.049719,0.001068,0.000053,"(200, 200, 200)","{'hidden_layer_sizes': (200, 200, 200)}",-1.872089e+07,-3.101676e+07,-1.273904e+07,-2.105624e+07,...,-2.060747e+07,5.941440e+06,3,-1.697089e+07,-1.446951e+07,-1.841590e+07,-1.821543e+07,-1.725251e+07,-1.706485e+07,1.409183e+06
9,2.869999,0.319238,0.001165,0.000142,"(1000, 100, 10)","{'hidden_layer_sizes': (1000, 100, 10)}",-2.479300e+07,-3.914060e+07,-3.227676e+07,-2.511212e+07,...,-2.889670e+07,6.047475e+06,4,-2.501002e+07,-1.955273e+07,-2.934452e+07,-2.338332e+07,-2.124366e+07,-2.370685e+07,3.373731e+06
8,32.032807,3.011274,0.005867,0.000487,"(800, 700, 600, 500, 400, 300, 200, 100)","{'hidden_layer_sizes': (800, 700, 600, 500, 40...",-1.791942e+07,-3.971804e+07,-3.960298e+07,-2.635563e+07,...,-2.900236e+07,9.160149e+06,5,-9.884797e+05,-5.714538e+05,-1.566739e+06,-5.439147e+05,-6.410590e+05,-8.623292e+05,3.865394e+05
2,0.909948,0.006428,0.000905,0.000030,"(100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100)}",-2.520822e+07,-3.999251e+07,-2.629018e+07,-2.920496e+07,...,-2.989490e+07,5.284894e+06,6,-2.440510e+07,-2.099525e+07,-2.506494e+07,-2.513934e+07,-2.603111e+07,-2.432715e+07,1.744425e+06
5,1.237142,0.019947,0.000794,0.000115,"(200, 200)","{'hidden_layer_sizes': (200, 200)}",-6.327808e+07,-6.126068e+07,-6.444263e+07,-5.036499e+07,...,-6.199717e+07,6.668754e+06,7,-5.739243e+07,-5.327603e+07,-5.585644e+07,-6.139151e+07,-5.335935e+07,-5.625515e+07,3.003177e+06
1,0.642481,0.020932,0.000871,0.000130,"(100, 100)","{'hidden_layer_sizes': (100, 100)}",-9.745093e+07,-9.729750e+07,-6.002519e+07,-5.991394e+07,...,-8.987762e+07,2.831926e+07,8,-8.683199e+07,-8.497857e+07,-9.775006e+07,-8.724492e+07,-8.659754e+07,-8.868062e+07,4.599519e+06
4,0.440291,0.003753,0.000720,0.000029,"(200,)","{'hidden_layer_sizes': (200,)}",-3.013237e+08,-3.886955e+08,-2.102267e+08,-2.159692e+08,...,-2.934938e+08,7.169487e+07,9,-2.913843e+08,-2.699043e+08,-3.148960e+08,-3.122993e+08,-2.786475e+08,-2.934263e+08,1.784942e+07
0,0.345393,0.009715,0.000769,0.000306,"(100,)","{'hidden_layer_sizes': (100,)}",-3.054345e+08,-3.944775e+08,-2.136767e+08,-2.198120e+08,...,-2.977657e+08,7.236567e+07,10,-2.955580e+08,-2.737516e+08,-3.191522e+08,-3.174670e+08,-2.829027e+08,-2.977663e+08,1.815481e+07


In [12]:
#tp3
tp3_window_size = 3600 # 1 hour window size
tp3_start_ts = 1422849600
tp3_end_ts = tp3_window_size * math.ceil(globalMaxTs/(tp3_window_size*1.0))
features,labels = getFeatures(tp3_start_ts,tp3_end_ts,tp3_window_size)
save_object(features,"q13_tp3_features")
save_object(labels,"q13_tp3_labels")

print("Finished Building feature vectors for time period 3")

Finished Building feature vectors for time period 3


In [13]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

hidden_layer_sizes = [(100,),
                      (100,100,),
                      (100,100,100,),
                      (100,100,100,100,100),
                      (200,),
                      (200,200,),
                      (200,200,200,),
                      (500,250,125,63,),
                      (800,700,600,500,400,300,200,100,),
                      (1000,100,10),                      
                     ]

param_grid = [
    {
        'hidden_layer_sizes' : hidden_layer_sizes,
    }
]


# nn = MLPRegressor(hidden_layer_sizes=(50,50), solver='adam')
nn = MLPRegressor(hidden_layer_sizes=(50,50),max_iter=500)

gs = GridSearchCV(estimator=nn,
                  param_grid=param_grid,
                  cv = KFold(5, shuffle=True), scoring='neg_mean_squared_error')

gs = gs.fit(scaled_features, labels)

GridSearch_table_plot(gs, "hidden_layer_sizes", negative=False)

save_object(gs,"q13_tp3_grid")

/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarnin

/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarnin

best parameters: {'hidden_layer_sizes': (800, 700, 600, 500, 400, 300, 200, 100)}
best score:      -2184253.66436 (+/-3682750.66870)
{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (800, 700, 600, 500, 400, 300, 200, 100),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}


/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/rishabketandoshi/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
8,31.364598,5.900373,0.008898,0.005221,"(800, 700, 600, 500, 400, 300, 200, 100)","{'hidden_layer_sizes': (800, 700, 600, 500, 40...",-2.517970e+05,-5.200641e+05,-4.320473e+05,-9.545577e+06,...,-2.184254e+06,3.682751e+06,1,-6.437505e+03,-6.772514e+03,-1.223184e+04,-8.204434e+03,-1.433069e+04,-9.595396e+03,3.138457e+03
3,2.609487,0.599917,0.002946,0.003425,"(100, 100, 100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100, 100, 100)}",-4.149507e+05,-5.139667e+05,-8.989521e+05,-1.002743e+07,...,-2.514449e+06,3.760199e+06,2,-2.752208e+04,-2.427440e+04,-4.596815e+04,-9.016063e+04,-1.382675e+05,-6.523855e+04,4.341953e+04
7,8.020208,2.028884,0.003171,0.002621,"(500, 250, 125, 63)","{'hidden_layer_sizes': (500, 250, 125, 63)}",-4.016623e+05,-3.788075e+05,-1.989173e+06,-9.977763e+06,...,-2.890249e+06,3.604241e+06,3,-2.280217e+05,-1.826768e+05,-1.997809e+05,-9.945649e+04,-5.068005e+05,-2.433473e+05,1.385106e+05
2,1.137332,0.036757,0.000891,0.000191,"(100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100)}",-2.782036e+05,-7.752335e+05,-8.629537e+05,-1.027273e+07,...,-3.133199e+06,3.740480e+06,4,-2.563459e+06,-2.508935e+06,-2.419913e+06,-2.813939e+05,-2.330821e+06,-2.020904e+06,8.733489e+05
9,3.942700,0.811978,0.001310,0.000058,"(1000, 100, 10)","{'hidden_layer_sizes': (1000, 100, 10)}",-2.776120e+05,-7.703432e+05,-8.694127e+05,-1.029186e+07,...,-3.217691e+06,3.758442e+06,5,-2.626161e+06,-2.513602e+06,-2.543878e+06,-2.954126e+05,-2.411618e+06,-2.078134e+06,8.940013e+05
5,1.657765,0.098660,0.001093,0.000015,"(200, 200)","{'hidden_layer_sizes': (200, 200)}",-2.893851e+05,-1.073852e+06,-8.557091e+05,-9.647452e+06,...,-3.233171e+06,3.500309e+06,6,-2.771749e+06,-2.650714e+06,-2.697280e+06,-4.845735e+05,-2.492937e+06,-2.219451e+06,8.722311e+05
1,0.702438,0.025803,0.000879,0.000207,"(100, 100)","{'hidden_layer_sizes': (100, 100)}",-2.883336e+05,-1.409460e+06,-8.913144e+05,-9.608997e+06,...,-3.352159e+06,3.459199e+06,7,-3.124317e+06,-2.883507e+06,-3.073961e+06,-8.240298e+05,-2.618647e+06,-2.504892e+06,8.589662e+05
6,86.260619,166.111403,0.003496,0.004433,"(200, 200, 200)","{'hidden_layer_sizes': (200, 200, 200)}",-2.678270e+05,-5.604045e+05,-9.412961e+05,-1.040269e+07,...,-3.704809e+06,4.030023e+06,8,-2.415990e+06,-2.233507e+06,-2.245324e+06,-1.682241e+05,-2.878155e+06,-1.988240e+06,9.395345e+05
4,0.500169,0.012581,0.000795,0.000013,"(200,)","{'hidden_layer_sizes': (200,)}",-3.609648e+06,-7.700289e+06,-3.048816e+06,-1.702949e+07,...,-7.106727e+06,5.220715e+06,9,-7.911270e+06,-6.947758e+06,-8.059327e+06,-4.661603e+06,-7.864379e+06,-7.088867e+06,1.275232e+06
0,0.434458,0.051651,0.000649,0.000220,"(100,)","{'hidden_layer_sizes': (100,)}",-3.983354e+06,-8.611317e+06,-3.348802e+06,-1.776831e+07,...,-7.681455e+06,5.366062e+06,10,-8.516966e+06,-7.555381e+06,-8.735266e+06,-5.131428e+06,-8.349712e+06,-7.657751e+06,1.324454e+06


In [14]:
GridSearch_table_plot(gs, "hidden_layer_sizes", negative=False)

best parameters: {'hidden_layer_sizes': (800, 700, 600, 500, 400, 300, 200, 100)}
best score:      -2184253.66436 (+/-3682750.66870)
{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (800, 700, 600, 500, 400, 300, 200, 100),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_iter': 500,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
8,31.364598,5.900373,0.008898,0.005221,"(800, 700, 600, 500, 400, 300, 200, 100)","{'hidden_layer_sizes': (800, 700, 600, 500, 40...",-2.517970e+05,-5.200641e+05,-4.320473e+05,-9.545577e+06,...,-2.184254e+06,3.682751e+06,1,-6.437505e+03,-6.772514e+03,-1.223184e+04,-8.204434e+03,-1.433069e+04,-9.595396e+03,3.138457e+03
3,2.609487,0.599917,0.002946,0.003425,"(100, 100, 100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100, 100, 100)}",-4.149507e+05,-5.139667e+05,-8.989521e+05,-1.002743e+07,...,-2.514449e+06,3.760199e+06,2,-2.752208e+04,-2.427440e+04,-4.596815e+04,-9.016063e+04,-1.382675e+05,-6.523855e+04,4.341953e+04
7,8.020208,2.028884,0.003171,0.002621,"(500, 250, 125, 63)","{'hidden_layer_sizes': (500, 250, 125, 63)}",-4.016623e+05,-3.788075e+05,-1.989173e+06,-9.977763e+06,...,-2.890249e+06,3.604241e+06,3,-2.280217e+05,-1.826768e+05,-1.997809e+05,-9.945649e+04,-5.068005e+05,-2.433473e+05,1.385106e+05
2,1.137332,0.036757,0.000891,0.000191,"(100, 100, 100)","{'hidden_layer_sizes': (100, 100, 100)}",-2.782036e+05,-7.752335e+05,-8.629537e+05,-1.027273e+07,...,-3.133199e+06,3.740480e+06,4,-2.563459e+06,-2.508935e+06,-2.419913e+06,-2.813939e+05,-2.330821e+06,-2.020904e+06,8.733489e+05
9,3.942700,0.811978,0.001310,0.000058,"(1000, 100, 10)","{'hidden_layer_sizes': (1000, 100, 10)}",-2.776120e+05,-7.703432e+05,-8.694127e+05,-1.029186e+07,...,-3.217691e+06,3.758442e+06,5,-2.626161e+06,-2.513602e+06,-2.543878e+06,-2.954126e+05,-2.411618e+06,-2.078134e+06,8.940013e+05
5,1.657765,0.098660,0.001093,0.000015,"(200, 200)","{'hidden_layer_sizes': (200, 200)}",-2.893851e+05,-1.073852e+06,-8.557091e+05,-9.647452e+06,...,-3.233171e+06,3.500309e+06,6,-2.771749e+06,-2.650714e+06,-2.697280e+06,-4.845735e+05,-2.492937e+06,-2.219451e+06,8.722311e+05
1,0.702438,0.025803,0.000879,0.000207,"(100, 100)","{'hidden_layer_sizes': (100, 100)}",-2.883336e+05,-1.409460e+06,-8.913144e+05,-9.608997e+06,...,-3.352159e+06,3.459199e+06,7,-3.124317e+06,-2.883507e+06,-3.073961e+06,-8.240298e+05,-2.618647e+06,-2.504892e+06,8.589662e+05
6,86.260619,166.111403,0.003496,0.004433,"(200, 200, 200)","{'hidden_layer_sizes': (200, 200, 200)}",-2.678270e+05,-5.604045e+05,-9.412961e+05,-1.040269e+07,...,-3.704809e+06,4.030023e+06,8,-2.415990e+06,-2.233507e+06,-2.245324e+06,-1.682241e+05,-2.878155e+06,-1.988240e+06,9.395345e+05
4,0.500169,0.012581,0.000795,0.000013,"(200,)","{'hidden_layer_sizes': (200,)}",-3.609648e+06,-7.700289e+06,-3.048816e+06,-1.702949e+07,...,-7.106727e+06,5.220715e+06,9,-7.911270e+06,-6.947758e+06,-8.059327e+06,-4.661603e+06,-7.864379e+06,-7.088867e+06,1.275232e+06
0,0.434458,0.051651,0.000649,0.000220,"(100,)","{'hidden_layer_sizes': (100,)}",-3.983354e+06,-8.611317e+06,-3.348802e+06,-1.776831e+07,...,-7.681455e+06,5.366062e+06,10,-8.516966e+06,-7.555381e+06,-8.735266e+06,-5.131428e+06,-8.349712e+06,-7.657751e+06,1.324454e+06
